# Automated ML


In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep



In [ ]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'puru-auto-experiment'

experiment=Experiment(ws, experiment_name)

In [ ]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
# create or attach an existing compute cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# set name of CPU cluster
cpu_cluster_name = "puru-compute-new"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, using it.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# wait for completion
cpu_cluster.wait_for_completion(show_output=True)

## Dataset

### Overview
I am using external data source from Kaggle. I have already downloaded the data from Kaggle and uploaded in my github project, so that we can give the data source as url. 
The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. The target (label) is "Outcome" column with values of 1 means has diabetes and 0 means no diabetes. We will be using all independent varialbles from the datasets like: preganicies the patient has had, their BMI, insulin level, age and so on.

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
# NOTE: update the key to match the dataset name
found = False
key = "diabetes-classification"
description_text = "Diabetes datasets for capstone project"

if key in ws.datasets.keys(): 
        found = True        
        dataset = ws.datasets[key] 

if not found:
        # Create Hyperdrive Dataset and register it into Workspace
        url = 'https://raw.githubusercontent.com/purunep/Capstoneproject/main/project/data/diabetes.csv'
        dataset = Dataset.Tabular.from_delimited_files(url)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
# prepare dataset for model training
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Clean dataset using the clean_data function
x, y = clean_data(dataset)

# split dataset into train and test sets
(x_train, x_test, y_train, y_test) = train_test_split(x, y, test_size= 0.2, random_state = 0)
label = 'Outcome'

# merge the output x and y dataframes into a single table for AutoML experiment
train_data_df = pd.concat([x_train, y_train], axis=1)

train_data_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
603,7,150,78,29,126,35.2,0.692,54,1
118,4,97,60,23,0,28.2,0.443,22,0
247,0,165,90,33,680,52.3,0.427,23,0
157,1,109,56,21,135,25.2,0.833,23,0
468,8,120,0,0,0,30.0,0.183,38,1


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory
# save training data in tabular format to allow for remote run
if not os.path.isdir('data'):  # create data folder if it does not exist
    os.mkdir('data')
    
if not os.path.exists('project_folder'):  # create project folder if it does not exist
    os.makedirs('project_folder')

# Save the train data to a csv file to be uploaded to the datastore
pd.DataFrame(train_data_df).to_csv("data/train_data.csv", index=False)

# Upload the training data as a tabular dataset for access during training on remote compute
# upload to data store
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='automlclassifier', overwrite=True, show_progress=True)

 # access datastore during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('automlclassifier/train_data.csv'))

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

I set enabled early stopping to True, to enable early termination if the score is not improving in the short term. Also set the iteration_timeout_minutes = 5, to set the maximum time in minutes that each iteration can run for before it terminates.Set max_concurrent_iterations = 4, to specify the maximum number of iterations that would be executed in parallel. Set max_cores_per_iteration = -1, to specify the maximum number of threads to use for a given training iteration. -1 means to use all the possible cores per iteration per child-run and set featurization = auto, to specify wherether featurization should be done automically or not, auto is ued to do it automatically

In [10]:
project_folder = './puruautoml'
automl_settings = {
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,    
    "featurization": 'auto'
}

# define automl autconfig parameters
automl_config = AutoMLConfig(
    experiment_timeout_minutes= 30,
    task= 'classification',
    primary_metric='accuracy',
    enable_onnx_compatible_models=True,
    compute_target=cpu_cluster, # included to allow for remote compute
    training_data= train_data,
    label_column_name= label,
    path = project_folder,
    n_cross_validations=3,
    debug_log = "automl_errors.log",    
    **automl_settings)

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

# wait for completion
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

********************************

{'runId': 'AutoML_2a8d0b39-7d5a-4b61-93b2-fc1cfcd64004',
 'target': 'puru-compute-new',
 'status': 'Completed',
 'startTimeUtc': '2020-12-30T21:38:39.351135Z',
 'endTimeUtc': '2020-12-30T22:02:11.77985Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'puru-compute-new',
  'AMLSettingsJsonString': '{"path":null,"name":"puru-auto-experiment","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group":"aml-quickstarts-132551","workspace_name":"quick-starts-ws-132551","region":"southcentralus","compute_target":"puru-compute-new","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
# Retrieve the best automl run model
best_automl_run, fitted_automl_model = remote_run.get_output()
print(best_automl_run)

# get best model and display properties
model_name = best_automl_run.properties['model_name']
print("Best Model:", model_name)

# display all the properties of the best model
#best_automl_run #.properties
best_automl_run.get_properties()

Run(Experiment: puru-auto-experiment,
Id: AutoML_2a8d0b39-7d5a-4b61-93b2-fc1cfcd64004_36,
Type: azureml.scriptrun,
Status: Completed)
Best Model: AutoML2a8d0b39736


{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'puru-auto-experiment\',\'compute_target\':\'puru-compute-new\',\'subscription_id\':\'61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_2a8d0b39-7d5a-4b61-93b2-fc1cfcd64004_36","experiment_name":"puru-auto-experiment","workspace_name":"quick-starts-ws-132551","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group_name":"aml-quickstarts-132551"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '36',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azurem

In [14]:
import joblib
joblib.dump(fitted_automl_model, 'automlmodel.pkl')

['automlmodel.pkl']

In [32]:
# register best autoML model for future deployment
from azureml.core.model import Model
model= Model.register(workspace = ws, model_name='automlmodel', model_path='automlmodel.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model automlmodel
automlmodel	automlmodel:2	2


In [33]:
#TODO: Save the best model
os.makedirs('outputs', exist_ok=True)
best_automl_run.download_file('outputs/model.pkl', 'outputs/automlmodel.pkl')
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score_aml.py')
best_automl_run.download_file('automl_driver.py', 'outputs/automl_driver.py')

In [34]:
import pickle
file = open("outputs/automlmodel.pkl",'rb')
aml_model = pickle.load(file)
file.close()

In [37]:
from train import clean_data
from sklearn.metrics import accuracy_score
import pandas as pd


factory = TabularDatasetFactory()
test_data_path = "https://raw.githubusercontent.com/purunep/Capstoneproject/main/project/data/diabetes.csv"
test_ds = pd.read_csv(test_data_path)
y_test = test_ds[['Outcome']]
print(accuracy_score(aml_model.predict(test_ds.drop(columns=['Outcome'])), y_test))

0.8619791666666666


In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test, aml_model.predict(test_ds.drop(columns=['Outcome']))))

              precision    recall  f1-score   support

           0       0.87      0.92      0.90       500
           1       0.84      0.75      0.79       268

    accuracy                           0.86       768
   macro avg       0.86      0.84      0.84       768
weighted avg       0.86      0.86      0.86       768



In [39]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ="automlmodel.pkl",
                        model_name = "automl_puru")

Registering model automl_puru


In [40]:

print(model.name)

automl_puru


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

In [57]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib
from azureml.core.model import Model

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    #mypath = os.getenv('AZUREML_MODEL_DIR')
    #print('puru my path: ' + mypath)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'automlmodel.pkl')
    
    model = joblib.load(model_path)
    #model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    #model_path = Model.get_model_path('automlbestmodel')
    #print('puru: ' + model_path)
    #model = joblib.load(model_path + model_path + '/automlbestmodel.pkl')
    

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Overwriting score.py


In [42]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               enable_app_insights=True, auth_enabled=True,
                                               tags={"data": "health",  "method" : "automl"}, 
                                               description='Predict diabetec with AutoML')

In [43]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('tutorial-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1'])

env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200821.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "tutorial-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"

In [55]:
%%writefile env.yml
channels:
- anaconda
- conda-forge
dependencies:
- pip
- python=3.6.2
- pip:
  - azureml-core==1.19.0
  - azureml-defaults==1.19.0
  - azureml-telemetry==1.19.0
  - azureml-train-restclients-hyperdrive==1.19.0
  - azureml-train-core==1.19.0
  - azureml-widgets==1.19.0
  - azureml-pipeline-core==1.19.0
  - azureml-pipeline-steps==1.19.0
  - azureml-opendatasets==1.19.0
  - azureml-automl-core==1.19.0
  - azureml-automl-runtime==1.19.0
  - azureml-train-automl-client==1.19.0
  - azureml-train-automl-runtime==1.19.0
  - azureml-train-automl==1.19.0
  - azureml-train==1.19.0
  - azureml-sdk==1.19.0
  - azureml-interpret==1.19.0
  - azureml-tensorboard==1.19.0
  - azureml-mlflow==1.19.0
  - mlflow
  - sklearn-pandas
  - scikit-learn
- pandas
- numpy
- tqdm
- py-xgboost
- joblib
- xgboost
- scikit-learn
- matplotlib
name: azureml_b1836710baabf05d37337a101fa8d6d1

Writing env.yml


In [59]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


env = Environment.from_conda_specification(name='myenv',file_path = 'env.yml')

# for pip_package in ["scikit-learn", "py-xgboost", "xgboost"]:
#     env.python.conda_dependencies.add_pip_package(pip_package)

# env.python.conda_dependencies("xgboost")

inference_config = InferenceConfig(entry_script='./score.py',
                                    environment=env)

In [60]:
f = open("envnew.yml", "w")
f.write(env.python.conda_dependencies.serialize_to_string())
f.close()

print("packages", env.python.conda_dependencies.serialize_to_string())

packages channels:
- anaconda
- conda-forge
dependencies:
- pip
- python=3.6.2
- pip:
  - azureml-core==1.19.0
  - azureml-defaults==1.19.0
  - azureml-telemetry==1.19.0
  - azureml-train-restclients-hyperdrive==1.19.0
  - azureml-train-core==1.19.0
  - azureml-widgets==1.19.0
  - azureml-pipeline-core==1.19.0
  - azureml-pipeline-steps==1.19.0
  - azureml-opendatasets==1.19.0
  - azureml-automl-core==1.19.0
  - azureml-automl-runtime==1.19.0
  - azureml-train-automl-client==1.19.0
  - azureml-train-automl-runtime==1.19.0
  - azureml-train-automl==1.19.0
  - azureml-train==1.19.0
  - azureml-sdk==1.19.0
  - azureml-interpret==1.19.0
  - azureml-tensorboard==1.19.0
  - azureml-mlflow==1.19.0
  - mlflow
  - sklearn-pandas
  - scikit-learn
- pandas
- numpy
- tqdm
- py-xgboost
- joblib
- xgboost
- scikit-learn
- matplotlib
name: azureml_b1836710baabf05d37337a101fa8d6d1



In [61]:
from azureml.core.webservice import AciWebservice,Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,enable_app_insights = True)
service = Model.deploy(ws, "autodeploy1", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [52]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'automl_puru')
print(model)

myenv = Environment.get(workspace=ws, name="tutorial-env", version="1")
inference_config = InferenceConfig(entry_script="autoscore.py", environment=myenv)

service_name = 'automlsvctwo'
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Model(workspace=Workspace.create(name='quick-starts-ws-132551', subscription_id='61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30', resource_group='aml-quickstarts-132551'), name=automl_puru, id=automl_puru:1, version=1, tags={}, properties={})
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................................................................................................................................................................................................................................................................................................................................
TimedOut


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 32f20b85-4082-4346-9fdb-3b0bd41ee147
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice. \nPlease refer to https://aka.ms/debugimage#dockerlog for more information."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 32f20b85-4082-4346-9fdb-3b0bd41ee147
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice. \nPlease refer to https://aka.ms/debugimage#dockerlog for more information."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Transitioning\nOperation ID: 32f20b85-4082-4346-9fdb-3b0bd41ee147\nMore inform

In [62]:
servicelogs = service.get_logs()
print(servicelogs)
#for line in servicelogs.split('\n'):
    #print(line)

2020-12-30T23:50:39,532005000+00:00 - gunicorn/run 
2020-12-30T23:50:39,531502100+00:00 - iot-server/run 
2020-12-30T23:50:39,533414800+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_afca42347aeb1ef9abd9ee384fe31fec/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_afca42347aeb1ef9abd9ee384fe31fec/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_afca42347aeb1ef9abd9ee384fe31fec/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_afca42347aeb1ef9abd9ee384fe31fec/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-12-30T23:50:39,545084000+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_afca42347aeb1ef9abd9ee384fe31fec/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
print(service.state)
print("scoring URI: " + service.scoring_uri)

print("Swagger URI: " + service.swagger_uri)

print("Authetication Key: " + service.get_keys())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
import json

input_payload = json.dumps({
    'data': dataset_x[0:2].tolist(),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)